# **<u> Aprendizaje Automático  </u>**

In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('./data/Student_performance_data.csv')
df

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,3388,18,1,0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
2388,3389,17,0,0,1,7.583217,4,1,4,0,1,0,0,3.279150,4.0
2389,3390,16,1,0,2,6.805500,20,0,2,0,0,0,1,1.142333,2.0
2390,3391,16,1,1,0,12.416653,17,0,2,0,1,1,0,1.803297,1.0


## **<u> Regresión (Predecir un número cuantitativo - entero / continuo) </u>**

In [288]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error


from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import Ridge

from sklearn.linear_model import Lasso

from sklearn.neighbors import KNeighborsRegressor

from sklearn.tree import DecisionTreeRegressor


In [289]:
def compare_models(model_dict):
    results = []

    for name, (mse, r2, mae, rmse, pred) in model_dict.items():
        stats = [name, mse, r2, mae, rmse, pred]
        results.append(stats)
    # df_results = pd.DataFrame(results, columns=['Modelo', 'Error Cuadrático Medio (MSE)', 'Coeficiente de Determinación (R^2)', 'Predicciones'])
    df_results = pd.DataFrame(results, columns=['Modelo', 'MSE', 'R2', 'MAE', 'RMSE', 'Predicciones'])
    return df_results

def gen_graph(x, y_pred, x_name, y_name, title):
    plt.figure(figsize=(14,7))

    sns.scatterplot(data=df, x=x_name, y=y_name)
    plt.plot(x, y_pred, color='red', label='Recta de regresión')
    plt.title(title)
    plt.savefig('./graph/' + title + '.png')
    print(y_pred.dtype)


### Regresion Lineal

In [290]:

# Función de regresión lineal
def regresion_lineal(x, y, x_n, y_n):
    # Crear un modelo de regresión lineal
    model = LinearRegression()

    # Ajustar el modelo a los datos
    model.fit(x, y)

    # Hacer predicciones para todos los valores de X
    y_pred = model.predict(x)

    # Calcular métricas
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    rmse = mean_squared_error(y, y_pred, squared=False)  # squared=False returns the RMSE

    # Generar gráfico
    gen_graph(x, y_pred, x_n, y_n, 'Modelo de regresión lineal para predecir los resultados')
    return mse, r2, mae, rmse, y_pred

### Regresión Polinómica

In [291]:
def regresion_polinomica(x, y, x_n, y_n):
    # Crear características polinómicas
    poly = PolynomialFeatures(degree=10)  # Cambiar degree para ajustar la complejidad del modelo
    x_poly = poly.fit_transform(x)

    # Crear el modelo de regresión polinómica
    poly_model = LinearRegression()

    # Ajustar el modelo a los datos
    poly_model.fit(x_poly, y)

    # Hacer predicciones
    y_poly_pred = poly_model.predict(x_poly)

    # Evaluar el modelo
    mse_poly = mean_squared_error(y, y_poly_pred)
    r2_poly = r2_score(y, y_poly_pred)
    mae_poly = mean_absolute_error(y, y_poly_pred)
    rmse_poly = mean_squared_error(y, y_poly_pred, squared=False)  # squared=False returns the RMSE

    # Obtenemos el valor mínimo y máximo de x
    x_min = x.min()
    x_max = x.max()

    # Creamos un rango de valores para x para dibujar la línea de regresión
    x_range = np.linspace(x_min, x_max, 100).reshape(-1, 1)
    x_range_poly = poly.transform(x_range)
    y_range_pred = poly_model.predict(x_range_poly)

    title = 'Modelo de regresión polinómica para predecir los resultados'

    plt.figure(figsize=(14,7))

    sns.scatterplot(data=df, x=x_n, y=y_n)
    plt.plot(x_range, y_range_pred, color='red', label='Recta de regresión')
    plt.title(title)
    plt.savefig('./graph/' + title + '.png')

    return mse_poly, r2_poly, mae_poly, rmse_poly, y_poly_pred

### Red Neuronal

In [292]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

def gen_graph_neuron(x, y, y_pred, x_n, y_n, title):
    plt.figure(figsize=(14, 7))

    # Graficar los puntos de datos originales
    sns.scatterplot(x=x.flatten(), y=y.flatten(), label='Datos')
    
    # Ordenar x y y_pred para una mejor visualización de la línea de predicción
    sorted_idx = np.argsort(x.flatten())
    x_sorted = x.flatten()[sorted_idx]
    y_pred_sorted = y_pred.flatten()[sorted_idx]
    
    # Graficar la línea de regresión neuronal
    plt.plot(x_sorted, y_pred_sorted, color='red', label='Regresión Neuronal')
    plt.xlabel(x_n)
    plt.ylabel(y_n)
    plt.legend()
    plt.title(title)
    plt.savefig('./graph/' + title + '.png')
    plt.show()

def regresion_neuronal(x, y, x_n, y_n):
    # Normalizar los datos de entrada
    x_mean = np.mean(x)
    x_std = np.std(x)
    y_mean = np.mean(y)
    y_std = np.std(y)

    x_norm = (x - x_mean) / x_std
    y_norm = (y - y_mean) / y_std

    # Dividir los datos en conjunto de entrenamiento y prueba
    x_train, x_test, y_train, y_test = train_test_split(x_norm, y_norm, test_size=0.2, random_state=42)

    # Crear el modelo de la red neuronal
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, input_dim=x_train.shape[1], activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    # Compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Entrenar el modelo
    history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=0)

    # Hacer predicciones
    y_pred = model.predict(x_norm)

    # Desnormalizar las predicciones
    y_pred = y_pred * y_std + y_mean

    # Evaluar el modelo
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    rmse = mean_squared_error(y, y_pred, squared=False)

    # Generar gráfico
    gen_graph_neuron(x, y, y_pred, x_n, y_n, 'Modelo de red neuronal para predecir los resultados')

    return mse, r2, mae, rmse, y_pred


In [293]:
x_n = 'GPA'
y_n = 'Absences'

x = pd.Series(df[x_n])
y = pd.Series(df[y_n])


# Asegurar que x tenga dos dimensiones (n_samples, n_features)
x = x.values.reshape(-1, 1)

# Asegurar que y tenga dos dimensiones (n_samples, 1)
y = y.values.reshape(-1, 1)

models = {'Regresión Lineal': regresion_lineal(x, y, x_n, y_n),
              'Regresión Polinomica': regresion_polinomica(x, y, x_n, y_n),
              'Regresión Neuronal': regresion_neuronal(x, y, x_n, y_n),
}

comparison = compare_models(models)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


float64


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
comparison

In [ ]:
import random

# Cargar una paleta de colores de seaborn
palette = sns.color_palette("husl", n_colors=len(comparison))

def get_color(index):
    return palette[index]

comparison = comparison.iloc[::-1]

plt.figure(figsize=(14, 7))
sns.scatterplot(data=df, x=x_n,  y=y_n, label='Datos originales')

# Agregar las predicciones de cada modelo
for index, row in comparison.iterrows():
    Modelo = row['Modelo']
    mse = row['MSE']
    r2 = row['R2']
    mae = row['MAE']
    rmse = row['RMSE']
    
    y_pred = row['Predicciones']
    # plt.plot(x, y_pred, color=random_color(), label=f'{Modelo} (MSE: {mse:.2f}, R2: {r2:.2f})')
    plt.plot(x, y_pred, color=get_color(index), label=f'{Modelo}')


plt.title('Comparación de Modelos de Regresión')
plt.xlabel('GPA')
plt.ylabel('Absences')
plt.legend()
plt.savefig('./graph/Comparacion_de_Modelos_de_Regression.png')
plt.show()

## **<u> Clasificación (Predecir una clase - binaria / multinomial) </u>**

### Regresión con Árbol de Decisión

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
def compare_models_class(model_dict):
    results = []
    confusion_matrices = {}

    for name, (confusion, exactitud, precision, sensibilidad, f1) in model_dict.items():
        stats = [name, exactitud, precision, sensibilidad, f1]
        results.append(stats)
        confusion_matrices[name] = confusion    

    df_results = pd.DataFrame(results, columns=['Modelo', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
    return df_results, confusion_matrices

def gen_graph_class(df_metricas):

    fig, ax = plt.subplots(2, 2, figsize=(14, 10))

    sns.barplot(x='Modelo', y='Accuracy', data=df_metricas, ax=ax[0, 0])
    ax[0, 0].set_title('Exactitud')

    sns.barplot(x='Modelo', y='Precision', data=df_metricas, ax=ax[0, 1])
    ax[0, 1].set_title('Precision')

    sns.barplot(x='Modelo', y='Recall', data=df_metricas, ax=ax[1, 0])
    ax[1, 0].set_title('Recall')

    sns.barplot(x='Modelo', y='F1 Score', data=df_metricas, ax=ax[1, 1])
    ax[1, 1].set_title('F1 Score')

    plt.tight_layout()
    plt.savefig('./graph/Modelos de Clasificacion.png')
    plt.show()

def gen_confusion_matrix(confusion_matrices):
    for model_name, confusion in confusion_matrices.items():
        plt.figure(figsize=(8, 6))
        sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Matriz de Confusión - {model_name}')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(f'./graph/Matriz de Confusión - {model_name}.png')
        plt.show()

In [ ]:
def random_forest(X_train, X_test, y_train, y_test):
    modelo = RandomForestClassifier()
    modelo.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = modelo.predict(X_test)  # Hacer predicciones
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return confusion, exactitud, precision, sensibilidad, f1

In [ ]:
def maquina_vectores_soporte(X_train, X_test, y_train, y_test):
    modelo = SVC()
    modelo.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = modelo.predict(X_test)  # Hacer predicciones
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return confusion, exactitud, precision, sensibilidad, f1

In [ ]:
def arbol_decision(X_train, X_test, y_train, y_test):
    modelo = DecisionTreeClassifier()
    modelo.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = modelo.predict(X_test)  # Hacer predicciones
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return confusion, exactitud, precision, sensibilidad, f1

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def red_neuronal_clasificacion(X_train, X_test, y_train, y_test):
    # Determinar el número de clases a partir de los datos
    num_classes = len(np.unique(y_train))
    input_dim = X_train.shape[1]
    
    # Convertir etiquetas a formato categórico (one-hot encoding)
    y_train_cat = to_categorical(y_train, num_classes=num_classes)
    y_test_cat = to_categorical(y_test, num_classes=num_classes)
    
    # Crear el modelo
    modelo = Sequential()
    modelo.add(Dense(64, input_dim=input_dim, activation='relu'))  # Capa oculta con 64 neuronas
    modelo.add(Dense(64, activation='relu'))  # Otra capa oculta con 64 neuronas
    modelo.add(Dense(num_classes, activation='softmax'))  # Capa de salida

    # Compilar el modelo
    modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrenar el modelo
    modelo.fit(X_train, y_train_cat, epochs=50, batch_size=10, verbose=1, validation_split=0.2)

    # Hacer predicciones
    y_pred_cat = modelo.predict(X_test)
    y_pred = y_pred_cat.argmax(axis=1)  # Convertir one-hot encoding a etiquetas

    # Calcular métricas
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return confusion, exactitud, precision, sensibilidad, f1


In [ ]:
def regresión_logistica(X_train, X_test, y_train, y_test):
    modelo = LogisticRegression()
    modelo.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = modelo.predict(X_test)  # Hacer predicciones
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return confusion, exactitud, precision, sensibilidad, f1

In [ ]:
def k_nearest(X_train, X_test, y_train, y_test):
    modelo = KNeighborsClassifier()
    modelo.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = modelo.predict(X_test)  # Hacer predicciones
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return confusion, exactitud, precision, sensibilidad, f1

In [ ]:
def naive_bayes(X_train, X_test, y_train, y_test):
    modelo = GaussianNB()
    modelo.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = modelo.predict(X_test)  # Hacer predicciones
    confusion = confusion_matrix(y_test, y_pred)
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    sensibilidad = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return confusion, exactitud, precision, sensibilidad, f1

In [ ]:
x_n = 'ParentalEducation'
y_n = 'GradeClass'

X = df[[x_n]].values
y = df[y_n].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

models = {
    'Random Forest': random_forest(X_train, X_test, y_train, y_test),
    'Máquina de Vectores de Soporte': maquina_vectores_soporte(X_train, X_test, y_train, y_test),
    'Árbol de Decisión': arbol_decision(X_train, X_test, y_train, y_test),
    'Red Neuronal' : red_neuronal_clasificacion(X_train, X_test, y_train, y_test),
    'Regresión Logística': regresión_logistica(X_train, X_test, y_train, y_test),
    'K-Nearest Neighbors': k_nearest(X_train, X_test, y_train, y_test),
    'Naive Bayes': naive_bayes(X_train, X_test, y_train, y_test)
}

comparison, confusion_matrices = compare_models_class(models)
gen_graph_class(comparison)
gen_confusion_matrix(confusion_matrices)
comparison